# **EKF Simulation**
In questo notebook implementeremo un ESEKF (Error State Extended Kalman Filter) avanzato con tecniche di correzione del coning, sculling delayed state.

In [36]:
import numpy as np
import math
import matplotlib

import utilities as tools
import circularBuffer

## Utilities

### LPF 1 ordine esponenziale

$$\alpha = exp(-\frac{2\pi f_c}{f_s})$$

$$y_k = \alpha y_{k-1} + (1 - \alpha)x_k$$

In [37]:
def calculate_alpha(fc, fs):  #fc = cutoff frequency | fs = sample frequency
    return math.exp(-((2*math.pi*fc) / (fs)))

def lpfFirstOrderExp(x, y_prev, alpha):    #x = new measure
    y = alpha * y_prev + (1 - alpha) * x
    lpfFirstOrderExp.y_prev = y

    return y

### LPF 2 ordine butterworth
$$y_k = b_0 x_k + b_1 x_{k-1} + b_2 x_{k-2} - a_1 y_{y-1} - a_2 y_{k-2}$$

### Coning Compensation

$$\Delta\theta _{k} = \omega dt$$
$$\Delta\theta _{corr} = \Delta\theta _{k-1} + \Delta\theta _{k} + \frac{1}{12}(\Delta\theta _{k-1} \times \Delta\theta _{k} )$$

In [38]:
def coning_compensation(omega, dtheta_prev, dt):
    dtheta = omega * dt

    return dtheta_prev + dtheta + 1/12*(np.cross(dtheta_prev, dtheta))

### Sculling Compensation


$$\Delta v_{k} = a dt$$
$$\Delta v_{corr} = \Delta v_{k-1} + \Delta v_{k} + \frac{1}{2}(\Delta\theta _{k-1} \times \Delta v_{k} ) + \frac{1}{12}(\Delta\theta _{k-1} \times \Delta v_{k-1} )$$

In [39]:
def sculling_compensation(acc, dv_prev, dtheta_prev, dt):
    dv = acc * dt

    return dv_prev + dv + 1/2*np.cross(dtheta_prev, dv) + 1/12*np.cross(dtheta_prev, dv_prev)

## EKF

$$\delta X = [\delta\theta, \delta p, \delta v, \delta b_g, \delta b_a ]   $$
$$\delta X ∈ ℝ^{15}

In [40]:
x_0 = np.array([1,0,0,0, 0,0,0, 0,0,0, 0,0,0, 0,0,0]).transpose()    # STATI NOMINALI ->  p[3] v[3] q[4] b_g[3] b_a[3]  forse-> earth magnetic field[3] body magnetic field [3]

delta_x_0 = np.array([0,0,0, 0,0,0, 0,0,0, 0,0,0, 0,0,0]).transpose()      #error states ->delta_pos[3], delta_vel[3],  delta_theta[3] (non si linearizzano i quaternioni)  delta_b_g[3] , delta_b_a[3]


g = 9.81





In [41]:


class EKF:
    def __init__(self, dt):
        
        sigma_acc = 0.03      # m/s^2 / sqrt(Hz)
        sigma_gyro = 0.001    # rad/s / sqrt(Hz)
        sigma_ba = 0.0001     # m/s^2 / sqrt(Hz)
        sigma_bg = 1e-6       # rad/s / sqrt(Hz)

        self.Q = np.diag([
            sigma_acc**2, sigma_acc**2, sigma_acc**2,       # accelerometro
            sigma_gyro**2, sigma_gyro**2, sigma_gyro**2,    # giroscopio
            sigma_ba**2, sigma_ba**2, sigma_ba**2,          # bias accel
            sigma_bg**2, sigma_bg**2, sigma_bg**2           # bias gyro
        ])

        sigma_gps_pos = 2
        sigma_gps_vel = 0.5
        #matrice di covarianza di misura del GPS
        self.R_gps = np.diag([
            sigma_gps_pos**2,sigma_gps_pos**2,sigma_gps_pos**2,
            sigma_gps_vel**2,sigma_gps_vel**2,sigma_gps_vel**2
        ])

        self.buffer_state = circularBuffer.CircularBuffer(math.ceil(1/2*dt))  #buffer per delayed state

        #Si inserisce il primo elemento del buffer
        t = 0 #timestamp
        x = np.array([0,0,0, 0,0,0, 1,0,0,0, 0,0,0, 0,0,0]).transpose() 
        P = np.zeros((15,15))

        self.buffer_state.add({
            "timestamp": t,
            "x_nominal": x.copy(),
            "P": P.copy(),
            "delta_theta_corr_prev": np.array([0,0,0]).transpose(),
            "delta_vel_corr_prev": np.array([0,0,0]).transpose(),
            "acc": np.array([0,0,0]).transpose(),
            "omega": np.array([0,0,0]).transpose()
        })

        self.propagation_step = 5   # Numero di propagazioni per ogni fase di predict
        self.propagation_index = 0  # indice del buffer a cui è arrivata la propagazione 
        self.propagation = False    # flag che indica se bisogna effettuare la propagazione


    def _jacobians_error_state(self, q, a, w, dt):
        """
        Restituisce F e G dell’error-state EKF2 di PX4.
        q : quaternion [qw, qx, qy, qz]
        a : accelero (bias corrected) in body frame
        w : gyro (bias corrected) in body frame
        dt: timestep
        """

        # --- rotation matrix ---
        R = tools.rotmat_body_to_ned(q)

        # Numero stati errore PX4
        n = 15

        # Matrici
        F = np.zeros((n, n))
        G = np.zeros((n, 12))   # rumori: accel, gyro, bias_a, bias_g

        # ---------------------------------------------------------
        # BLOCCO POSIZIONE
        # δp_dot = δv
        # ---------------------------------------------------------
        F[0:3, 3:6] = np.eye(3)

        # ---------------------------------------------------------
        # BLOCCO VELOCITÀ
        # δv_dot = -R[a]_× δθ  - R δb_a  + R n_a
        # ---------------------------------------------------------
        F[3:6, 6:9] = -R @ tools.skew(a)
        F[3:6, 9:12] = -R

        # rumore accelerometro
        G[3:6, 0:3] = R

        # ---------------------------------------------------------
        # BLOCCO ORIENTAZIONE
        # δθ_dot = -[ω]_× δθ - δb_g + n_g
        # ---------------------------------------------------------
        F[6:9, 6:9] = -tools.skew(w)
        F[6:9, 12:15] = -np.eye(3)

        # rumore gyro
        G[6:9, 3:6] = np.eye(3)

        # ---------------------------------------------------------
        # BLOCCO BIAS ACCELERO
        # δb_a_dot = n_ba
        # ---------------------------------------------------------
        # modello: random walk → F = 0, G = I * sigma
        G[9:12, 6:9] = np.eye(3)

        # ---------------------------------------------------------
        # BLOCCO BIAS GIRO
        # δb_g_dot = n_bg
        # ---------------------------------------------------------
        G[12:15, 9:12] = np.eye(3)

        # ---------------------------------------------------------
        # BLOCCO VENTO in PX4:
        # δw_dot = 0 (stato costante)
        # ---------------------------------------------------------
        # Nessuna dinamica → tutte le Jacobiane restano 0

        # ---------------------------------------------------------
        # Discretizzazione: Φ = I + F * dt
        # ---------------------------------------------------------
        Phi = np.eye(n) + F * dt

        # Noise discretization: Gd = G * dt
        Gd = G * dt

        return Phi, Gd, F, G
    
    def _predict_nominal_states(self, x_prev, delta_theta_corr, delta_vel_corr, dt):
        #Stima stati nominali
        x = np.empty((16,), dtype=float)

        x[6:10] =  tools.quat_update(x_prev[6:10], delta_theta_corr)
        x[3:6] = x_prev[3:6] + tools.rotmat_body_to_ned(x[6:10]) @ delta_vel_corr
        x[0:3] = x_prev[0:3] + x[3:6] * dt

        x[10:13] = x_prev[10:13] + np.random.normal(scale=1e-6, size=3)
        x[13:16] = x_prev[13:16] + np.random.normal(scale=1e-7, size=3)

        return x
    
    #funzione che manda avanti la propagazione
    def _propagation(self, dt):
        
        if self.propagation:
            for i in range(self.propagation_step):
                if self.propagation_index <= len(self.buffer_state.buffer):
                    buffer_element = self.buffer_state.get(self.propagation_index)

                    x_propagated = self._predict_nominal_states(self.buffer_state.get(self.propagation_index - 1).x_nominal , buffer_element.delta_theta_corr, buffer_element.delta_vel_corr, dt)

                    Phi, Gd, F, G = self._jacobians_error_state(buffer_element.x_nominal[6:10], buffer_element.acc, buffer_element.omega, dt)

                    P = Phi @ self.P_prev @ Phi.T + Gd @ self.Q @ Gd.T

                    self.buffer_state.update(self.propagation_index, x_propagated.copy(), P.copy())

                    self.propagation_index += 1
                else:
                    self.propagation = False


    def predict(self, gyro_meas, acc_meas, dt):

        #capire come gestire lo switching atomico     
        prev = self.buffer_state.get()  #si prende l'ultimo elemento del buffer => l'elemento stimato nel predict prima

        #correzione bias
        omega = gyro_meas - prev["x_nominal"][10:13]
        acc = acc_meas - prev["x_nominal"][13:16] + np.array([0,0,g]).transpose()

        #Compensation
        delta_theta_corr = coning_compensation(omega, prev["delta_theta_corr_prev"], dt)
        delta_vel_corr = sculling_compensation(acc, prev["delta_vel_corr_prev"], prev["delta_theta_corr_prev"], dt)

        x = self._predict_nominal_states(prev["x_nominal"], delta_theta_corr, delta_vel_corr, dt)

        self._propagation(dt)

        #Stima error state

        Phi, Gd, F, G = self._jacobians_error_state(x[6:10], acc, omega, dt)

        P = Phi @ prev["P"] @ Phi.T + Gd @ self.Q @ Gd.T

        t = 0 #timestamp
        self.buffer_state.add({
            "timestamp": t,
            "x_nominal": x.copy(),
            "P": P.copy(),
            "delta_theta_corr_prev": delta_theta_corr,
            "delta_vel_corr_prev": delta_vel_corr,
            "acc": acc,
            "omega": omega
        })

        return x, P
    
    # gps data -> spostamento rispetto a un punto e velocità nel NED frame
    # {
    #       "timestamp":x,
    #       "meas":pos[3] vel[3]
    #}
    
    def update_GPS(self, gps_data):
        gps_timestamp = gps_data.timestamp
        gps_meas = gps_data.meas
        #trovare elemento nel buffer
        buffer_state_old, index = self.buffer_state.get_by_timestamp(gps_timestamp)

        if buffer_state_old is None:
            return False    # => misura troppo vecchia

        x_nominal_old = buffer_state_old["x_nominal"].ravel()
        P_old = buffer_state_old["P"]

        ##

        innovation = gps_meas - x_nominal_old[0:6]

        H = np.zeros((6,15))
        H[0:3, 0:3] = np.eye(3)
        H[3:6, 3:6] = np.eye(3)

        S = H @ P_old @ H.T + self.R_gps      # matrice dell'innovazione di covarianza S
        K = P_old @ H.T @ np.linalg.inv(S)

        #calcolo delta_x -> correggo lo stato nominale con delta_x -> delta_x = 0

        delta_x = K*innovation

        x_updated = x_nominal_old.copy()
        x_updated[0:6] = x_nominal_old + delta_x[0:6]

        dtheta = delta_x[6:9]
        dq = tools.quat_from_small_angle(dtheta)

        x_updated[6:10] = tools.quat_multiply(dq, x_nominal_old[6:10])

        x_updated[10:13] = x_nominal_old + delta_x[10:13]
        x_updated[13:16] = x_nominal_old + delta_x[13:16]

        P_new = (np.eye(15) - K @ H) @ P_old @ (np.eye(15) - K @ H).T + K @ self.R_gps @ K.T #da cambiare con quella usata da PX4
 
        self.buffer_state.update(index, x_updated.copy(), P_new.copy())

        #mettere indice = indice elemento appena aggiornato + 1
        self.propagation_index = index + 1
        self.propagation = True


    def update_MAG(self, mag_data):
        pass

## Simulation

In [42]:
ekf = EKF(0.04)

for i in range(10):
    x, p = ekf.predict(np.array([0,0,0]).transpose(), np.array([0,0,981]).transpose(),0.04)

    print(x)
    print(p)
    print(ekf.buffer_state.__str__)

[            nan             nan             nan             nan
             nan             nan             nan             nan
             nan             nan  1.77841172e-07  1.93830995e-07
 -1.17727820e-06  1.21678385e-07  3.95430663e-08 -9.73495292e-09]
[[0.0e+00 0.0e+00 0.0e+00     nan     nan     nan 0.0e+00 0.0e+00 0.0e+00
  0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00]
 [0.0e+00 0.0e+00 0.0e+00     nan     nan     nan 0.0e+00 0.0e+00 0.0e+00
  0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00]
 [0.0e+00 0.0e+00 0.0e+00     nan     nan     nan 0.0e+00 0.0e+00 0.0e+00
  0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00]
 [    nan     nan     nan     nan     nan     nan     nan     nan     nan
      nan     nan     nan     nan     nan     nan]
 [    nan     nan     nan     nan     nan     nan     nan     nan     nan
      nan     nan     nan     nan     nan     nan]
 [    nan     nan     nan     nan     nan     nan     nan     nan     nan
      nan     nan     nan     nan     na

/home/reike/Desktop/scrivania/robotics/simulations&notebooks/ekf/utilities.py:41: RuntimeWarning: invalid value encountered in divide
  u = delta_theta / theta
